## import stuff

In [1]:
#importing things
import pandas as pd
import numpy as np
import csv
import pylab as pl
import matplotlib.pyplot as plt
import copy
import scipy
from scipy.stats import linregress
from scipy.stats import t
from scipy.stats import sem
from scipy.stats import chi2
from scipy.stats import ttest_ind_from_stats
from mpl_toolkits.mplot3d import Axes3D
from scipy.stats import ttest_ind_from_stats
from collections import OrderedDict
import seaborn as sns

## fetch the data

In [2]:
cells = pd.DataFrame(pd.read_csv('ch65i_cell_counts.csv',delimiter=','))
cells

,CH65_i_MI_neg_a,CH65_i_MI_pos_a,CH65_i_MI_neg_b,CH65_i_MI_pos_b,CH65_i_MI_neg_c,CH65_i_MI_pos_c,CH65_i_SI_G189D_neg_a,CH65_i_SI_G189D_pos_a,CH65_i_SI_G189D_neg_b,CH65_i_SI_G189D_pos_b,CH65_i_SI_G189D_neg_c,CH65_i_SI_G189D_pos_c
0,2942707,67100,2931639,61508,2916019,40503,2513389,451945,2454688,486049,2372675,553796


In [3]:
#read in data
counts=pd.DataFrame()
counts = pd.DataFrame(pd.read_csv('count_table.tsv',delimiter='\t',dtype={'geno': str}))
counts

,geno,CH65_i_MI_neg_a,CH65_i_MI_pos_a,CH65_i_MI_neg_b,CH65_i_MI_pos_b,CH65_i_MI_neg_c,CH65_i_MI_pos_c,CH65_i_SI_G189D_neg_a,CH65_i_SI_G189D_pos_a,CH65_i_SI_G189D_neg_b,CH65_i_SI_G189D_pos_b,CH65_i_SI_G189D_neg_c,CH65_i_SI_G189D_pos_c,CH65_i_unsorted
0,0000000000000000,416,6,369,4,278,9,270,1,322,1,292,1,224
1,0000000000000001,396,5,307,4,246,4,281,2,386,1,283,5,207
2,0000000000000010,666,10,511,2,393,22,358,0,664,7,506,4,293
3,0000000000000011,617,18,517,19,366,14,342,11,498,3,407,15,259
4,0000000000000100,946,7,697,11,503,15,526,0,816,11,781,11,355
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65531,1111111111111011,554,29,291,20,268,16,25,267,31,283,12,295,196
65532,1111111111111100,359,6,231,3,209,10,32,72,28,146,18,139,92
65533,1111111111111101,350,21,171,2,171,8,50,104,45,144,32,165,112
65534,1111111111111110,711,20,245,5,337,18,97,234,119,244,92,299,255


## Calculate fraction bound for each replicate and antigen

In [4]:
#for each variant infer number of cells in that bin using read fraction and cell counts
counts['CH65_i_MI_neg_a_norm'] = (counts['CH65_i_MI_neg_a']/counts['CH65_i_MI_neg_a'].sum())*cells['CH65_i_MI_neg_a'].mean()
counts['CH65_i_MI_neg_b_norm'] = (counts['CH65_i_MI_neg_b']/counts['CH65_i_MI_neg_b'].sum())*cells['CH65_i_MI_neg_b'].mean()
counts['CH65_i_MI_neg_c_norm'] = (counts['CH65_i_MI_neg_c']/counts['CH65_i_MI_neg_c'].sum())*cells['CH65_i_MI_neg_c'].mean()
counts['CH65_i_MI_pos_a_norm'] = (counts['CH65_i_MI_pos_a']/counts['CH65_i_MI_pos_a'].sum())*cells['CH65_i_MI_pos_a'].mean()
counts['CH65_i_MI_pos_b_norm'] = (counts['CH65_i_MI_pos_b']/counts['CH65_i_MI_pos_b'].sum())*cells['CH65_i_MI_pos_b'].mean()
counts['CH65_i_MI_pos_c_norm'] = (counts['CH65_i_MI_pos_c']/counts['CH65_i_MI_pos_c'].sum())*cells['CH65_i_MI_pos_c'].mean()
counts['CH65_i_SI_G189D_neg_a_norm'] = (counts['CH65_i_SI_G189D_neg_a']/counts['CH65_i_SI_G189D_neg_a'].sum())*cells['CH65_i_SI_G189D_neg_a'].mean()
counts['CH65_i_SI_G189D_neg_b_norm'] = (counts['CH65_i_SI_G189D_neg_b']/counts['CH65_i_SI_G189D_neg_b'].sum())*cells['CH65_i_SI_G189D_neg_b'].mean()
counts['CH65_i_SI_G189D_neg_c_norm'] = (counts['CH65_i_SI_G189D_neg_c']/counts['CH65_i_SI_G189D_neg_c'].sum())*cells['CH65_i_SI_G189D_neg_c'].mean()
counts['CH65_i_SI_G189D_pos_a_norm'] = (counts['CH65_i_SI_G189D_pos_a']/counts['CH65_i_SI_G189D_pos_a'].sum())*cells['CH65_i_SI_G189D_pos_a'].mean()
counts['CH65_i_SI_G189D_pos_b_norm'] = (counts['CH65_i_SI_G189D_pos_b']/counts['CH65_i_SI_G189D_pos_b'].sum())*cells['CH65_i_SI_G189D_pos_b'].mean()
counts['CH65_i_SI_G189D_pos_c_norm'] = (counts['CH65_i_SI_G189D_pos_c']/counts['CH65_i_SI_G189D_pos_c'].sum())*cells['CH65_i_SI_G189D_pos_c'].mean()

#for each antigen and replicate compute the percent bound
counts['MIa_frac'] = 100*counts['CH65_i_MI_pos_a_norm']/(counts['CH65_i_MI_pos_a_norm']+counts['CH65_i_MI_neg_a_norm'])
counts['MIb_frac'] = 100*counts['CH65_i_MI_pos_b_norm']/(counts['CH65_i_MI_pos_b_norm']+counts['CH65_i_MI_neg_b_norm'])
counts['MIc_frac'] = 100*counts['CH65_i_MI_pos_c_norm']/(counts['CH65_i_MI_pos_c_norm']+counts['CH65_i_MI_neg_c_norm'])
counts['SIa_frac'] = 100*counts['CH65_i_SI_G189D_pos_a_norm']/(counts['CH65_i_SI_G189D_pos_a_norm']+counts['CH65_i_SI_G189D_neg_a_norm'])
counts['SIb_frac'] = 100*counts['CH65_i_SI_G189D_pos_b_norm']/(counts['CH65_i_SI_G189D_pos_b_norm']+counts['CH65_i_SI_G189D_neg_b_norm'])
counts['SIc_frac'] = 100*counts['CH65_i_SI_G189D_pos_c_norm']/(counts['CH65_i_SI_G189D_pos_c_norm']+counts['CH65_i_SI_G189D_neg_c_norm'])


#for each antigen compute the mean and sem
counts['MI_frac_mean'] = counts[['MIa_frac','MIb_frac','MIc_frac']].mean(axis=1)
counts['SI_frac_mean'] = counts[['SIa_frac','SIb_frac','SIc_frac']].mean(axis=1)
counts['MI_frac_sem'] = counts[['MIa_frac','MIb_frac','MIc_frac']].sem(axis=1)
counts['SI_frac_sem'] = counts[['SIa_frac','SIb_frac','SIc_frac']].sem(axis=1)

counts

,geno,CH65_i_MI_neg_a,CH65_i_MI_pos_a,CH65_i_MI_neg_b,CH65_i_MI_pos_b,CH65_i_MI_neg_c,CH65_i_MI_pos_c,CH65_i_SI_G189D_neg_a,CH65_i_SI_G189D_pos_a,CH65_i_SI_G189D_neg_b,...,MIa_frac,MIb_frac,MIc_frac,SIa_frac,SIb_frac,SIc_frac,MI_frac_mean,SI_frac_mean,MI_frac_sem,SI_frac_sem
0,0000000000000000,416,6,369,4,278,9,270,1,322,...,1.803128,0.866825,0.888826,0.389859,0.388136,0.308175,1.186260,0.362057,0.308500,0.026945
1,0000000000000001,396,5,307,4,246,4,281,2,386,...,1.582050,1.040063,0.448404,0.746513,0.323990,1.569760,1.023506,0.880088,0.327360,0.365772
2,0000000000000010,666,10,511,2,393,22,358,0,664,...,1.875742,0.314716,1.527016,0.000000,1.305424,0.708505,1.239158,0.671309,0.473057,0.377302
3,0000000000000011,617,18,517,19,366,14,342,11,498,...,3.581133,2.879085,1.048493,3.287143,0.750153,3.219625,2.502904,2.418974,0.754917,0.834638
4,0000000000000100,946,7,697,11,503,15,526,0,816,...,0.933267,1.257027,0.819307,0.000000,1.663209,1.255383,1.003200,0.972864,0.131108,0.500476
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65531,1111111111111011,554,29,291,20,268,16,25,267,31,...,6.247984,5.252699,1.626889,91.860636,91.970379,95.687840,4.375857,93.172952,1.404192,1.257843
65532,1111111111111100,359,6,231,3,209,10,32,72,28,...,2.083454,1.036721,1.308072,70.393729,86.741249,87.453723,1.476082,81.529567,0.313626,5.571716
65533,1111111111111101,350,21,171,2,171,8,50,104,45,...,7.096654,0.934619,1.279376,68.730624,80.059550,82.314369,3.103549,77.034848,1.999031,4.202822
65534,1111111111111110,711,20,245,5,337,18,97,234,119,...,3.457406,1.619539,1.458009,71.825007,72.009126,74.578152,2.178318,72.804095,0.641242,0.888619


## functions for annotating dataframe

In [5]:
#define categories based on mutation position
def find_pos_1(variant):
    pos = [mut for mut in variant]
    if pos[0] == "1":
        return 1
    else:
        return 0
def find_pos_2(variant):
    pos = [mut for mut in variant]
    if pos[1] == "1":
        return 1
    else:
        return 0
def find_pos_3(variant):
    pos = [mut for mut in variant]
    if pos[2] == "1":
        return 1
    else:
        return 0
def find_pos_4(variant):
    pos = [mut for mut in variant]
    if pos[3] == "1":
        return 1
    else:
        return 0
def find_pos_5(variant):
    pos = [mut for mut in variant]
    if pos[4] == "1":
        return 1
    else:
        return 0
def find_pos_6(variant):
    pos = [mut for mut in variant]
    if pos[5] == "1":
        return 1
    else:
        return 0
def find_pos_7(variant):
    pos = [mut for mut in variant]
    if pos[6] == "1":
        return 1
    else:
        return 0
def find_pos_8(variant):
    pos = [mut for mut in variant]
    if pos[7] == "1":
        return 1
    else:
        return 0
def find_pos_9(variant):
    pos = [mut for mut in variant]
    if pos[8] == "1":
        return 1
    else:
        return 0
def find_pos_10(variant):
    pos = [mut for mut in variant]
    if pos[9] == "1":
        return 1
    else:
        return 0
def find_pos_11(variant):
    pos = [mut for mut in variant]
    if pos[10] == "1":
        return 1
    else:
        return 0
def find_pos_12(variant):
    pos = [mut for mut in variant]
    if pos[11] == "1":
        return 1
    else:
        return 0
def find_pos_13(variant):
    pos = [mut for mut in variant]
    if pos[12] == "1":
        return 1
    else:
        return 0
def find_pos_14(variant):
    pos = [mut for mut in variant]
    if pos[13] == "1":
        return 1
    else:
        return 0
def find_pos_15(variant):
    pos = [mut for mut in variant]
    if pos[14] == "1":
        return 1
    else:
        return 0
def find_pos_16(variant):
    pos = [mut for mut in variant]
    if pos[15] == "1":
        return 1
    else:
        return 0
    
#add a column with the number of mutations and save the file
def sum_digits(digit):
    return sum(int(x) for x in digit if x.isdigit())

## output the data

In [6]:
#define categories based on mutation position
counts['pos1'] = counts['geno'].apply(find_pos_1)
counts['pos2'] = counts['geno'].apply(find_pos_2)
counts['pos3'] = counts['geno'].apply(find_pos_3)
counts['pos4'] = counts['geno'].apply(find_pos_4)
counts['pos5'] = counts['geno'].apply(find_pos_5)
counts['pos6'] = counts['geno'].apply(find_pos_6)
counts['pos7'] = counts['geno'].apply(find_pos_7)
counts['pos8'] = counts['geno'].apply(find_pos_8)
counts['pos9'] = counts['geno'].apply(find_pos_9)
counts['pos10'] = counts['geno'].apply(find_pos_10)
counts['pos11'] = counts['geno'].apply(find_pos_11)
counts['pos12'] = counts['geno'].apply(find_pos_12)
counts['pos13'] = counts['geno'].apply(find_pos_13)
counts['pos14'] = counts['geno'].apply(find_pos_14)
counts['pos15'] = counts['geno'].apply(find_pos_15)
counts['pos16'] = counts['geno'].apply(find_pos_16)

#add a column with the number of mutations and save the file
counts['som_mut'] = counts['geno'].apply(sum_digits)
counts.to_csv('20220725_fracbound.csv', index=False)